In [1]:
import heapq
from collections import deque

class Node:
    def __init__(self, state, parent=None, path_cost=0, heuristic=0):
        self.state = state
        self.parent = parent
        self.path_cost = path_cost
        self.heuristic = heuristic

    def f_value(self):
        return self.path_cost + self.heuristic  # f(n) = g(n) + h(n)

    def __lt__(self, other):
        return self.f_value() < other.f_value()


In [2]:
def print_solution(node):
    path = []
    total_cost = node.path_cost
    while node:
        path.append(node.state)
        node = node.parent
    path.reverse()

    for state in path:
        print_state(state)
        print("->", end=" ")
    print(f"\nTotal path cost: {total_cost}")



In [3]:
def print_state(state):
    for i in range(0, 9, 3):
        print(' '.join(state[i:i+3]))
    print()



In [4]:
def heuristic(state):
    # Example heuristic: Manhattan distance
    goal_state = '123456780'
    distance = 0
    for i in range(9):
        if state[i] != '0':
            goal_index = goal_state.index(state[i])
            current_row, current_col = divmod(i, 3)
            goal_row, goal_col = divmod(goal_index, 3)
            distance += abs(current_row - goal_row) + abs(current_col - goal_col)
    return distance



In [5]:
def get_neighbors(state):
    # Find the position of the blank space '0'
    index = state.index('0')
    neighbors = []
    moves = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # Up, Down, Left, Right
    row, col = divmod(index, 3)

    for dr, dc in moves:
        new_row, new_col = row + dr, col + dc
        if 0 <= new_row < 3 and 0 <= new_col < 3:
            new_index = new_row * 3 + new_col
            new_state = list(state)
            new_state[index], new_state[new_index] = new_state[new_index], new_state[index]
            neighbors.append((''.join(new_state), 1))  # 1 is the cost of each move

    return neighbors



In [6]:
def a_star_search(initial_state, goal_state):
    # Create the initial node
    initial_node = Node(state=initial_state, heuristic=heuristic(initial_state))
    # Initialize the frontier with the initial node
    frontier = []
    heapq.heappush(frontier, initial_node)
    # Initialize the explored set
    explored = set()

    while frontier:
        # Pop the node with the lowest f-value
        node = heapq.heappop(frontier)

        # Check if the goal has been reached
        if node.state == goal_state:
            return print_solution(node)

        # Add the node's state to the explored set
        explored.add(node.state)

        # Expand the node
        for child_state, cost in get_neighbors(node.state):
            if child_state not in explored:
                child = Node(state=child_state, parent=node, path_cost=node.path_cost + cost, heuristic=heuristic(child_state))
                # If the child state is not in frontier, add it to the frontier
                if all(frontier_node.state != child_state for frontier_node in frontier):
                    heapq.heappush(frontier, child)
                # If the child state is in frontier with a higher f-value, replace it
                else:
                    for i in range(len(frontier)):
                        if frontier[i].state == child_state and frontier[i].f_value() > child.f_value():
                            frontier[i] = child
                            heapq.heapify(frontier)
                            break

    return "Failure: No solution found."


In [7]:
# Example usage
initial_state = '123405678'  # Initial state
goal_state = '123456780'     # Goal state

# Perform A* Search
solution = a_star_search(initial_state, goal_state)
print(solution)

1 2 3
4 0 5
6 7 8

-> 1 2 3
4 5 0
6 7 8

-> 1 2 3
4 5 8
6 7 0

-> 1 2 3
4 5 8
6 0 7

-> 1 2 3
4 5 8
0 6 7

-> 1 2 3
0 5 8
4 6 7

-> 1 2 3
5 0 8
4 6 7

-> 1 2 3
5 6 8
4 0 7

-> 1 2 3
5 6 8
4 7 0

-> 1 2 3
5 6 0
4 7 8

-> 1 2 3
5 0 6
4 7 8

-> 1 2 3
0 5 6
4 7 8

-> 1 2 3
4 5 6
0 7 8

-> 1 2 3
4 5 6
7 0 8

-> 1 2 3
4 5 6
7 8 0

-> 
Total path cost: 14
None
